<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/Neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"on device: {device}")

on device: cpu


In [ ]:
## Questions:

#Do I take the Conv output before or after ReLU

In [13]:
vgg_pretrained_model.features.0

SyntaxError: ignored

In [9]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

[INFO] Couldn't find torchinfo... installing it.


In [69]:
weights = torchvision.models.VGG19_Weights.IMAGENET1K_V1 # 	height=224, width=224
vgg_pretrained_model = torchvision.models.vgg19(weights=weights).to(device)
#Freeze weights
# for param in vgg_pretrained_model.features.parameters():
#       param.requires_grad = False

#Register hook
activations = {}
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

output_idxs = [0, 5, 10, 19, 28]
for conv_num, i in enumerate(output_idxs):
  vgg_pretrained_model.features[i].register_forward_hook(get_activation('conv'+str(conv_num+1)))

In [70]:
content = torch.rand((1, 3, 224, 224))
style = torch.rand((1, 3, 224, 224))
output = torch.rand((1, 3, 224, 224))
alpha, beta = 0.9, 0.1


input = torch.concat((content, style, output), dim=0)
_ = vgg_pretrained_model(input)

In [74]:
for key in activations: print(key, activations[key].shape)
content_outputs = []
style_outputs = []
generated_outputs = []
for key in activations:
  content_outputs.append(activations[key][0])
  style_outputs.append(activations[key][1])
  generated_outputs.append(activations[key][2])

content_loss = 0
for i in range(5): content_loss+= nn.functional.mse_loss(content_outputs[i], generated_outputs[i])

conv1 torch.Size([3, 64, 224, 224])
conv2 torch.Size([3, 128, 112, 112])
conv3 torch.Size([3, 256, 56, 56])
conv4 torch.Size([3, 512, 28, 28])
conv5 torch.Size([3, 512, 14, 14])


In [75]:
#Gram matrix for Style and generated Image

In [77]:
style_gram = []
style_outputs[0].shape

torch.Size([64, 224, 224])

In [49]:


]

content_loss= 2
style_loss = 2
total_loss = alpha*content_loss + beta*style_loss

# content_loss = 
print(total_loss)

2.0


In [50]:
content_outputs[0].shape, generated_outputs[0].shape

(torch.Size([64, 224, 224]), torch.Size([64, 224, 224]))

AttributeError: ignored

Sequential(
  (30): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (31): ReLU(inplace=True)
  (32): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (33): ReLU(inplace=True)
  (34): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (35): ReLU(inplace=True)
  (36): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [36]:
#Deleting end layers to reduce model computational overhead
vgg_pretrained_model.features[30:] = nn.Sequential()
vgg_pretrained_model.avgpool = nn.Sequential()
vgg_pretrained_model.classifier = nn.Sequential()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [10]:
summary(vgg_pretrained_model, input_size=[1, 3, 224, 224], col_names =['input_size', 'output_size', 'num_params', 'trainable'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [1, 3, 224, 224]          [1, 1000]                 --                        True
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 512, 7, 7]            --                        True
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 224, 224]         1,792                     True
│    └─ReLU: 2-2                         [1, 64, 224, 224]         [1, 64, 224, 224]         --                        --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         [1, 64, 224, 224]         36,928                    True
│    └─ReLU: 2-4                         [1, 64, 224, 224]         [1, 64, 224, 224]         --                        --
│    └─MaxPool2d: 2-5                    [1, 64, 224, 224]         [1, 64, 112, 112]         --                        --
│    └─Co